# Exemplo 8: Logística - Otimização de Rotas (Dask)

Este notebook demonstra o processamento de logs de entrega para identificar gargalos e otimizar rotas usando **Dask**.

**Cenário**: Analisar dados de GPS de caminhões de entrega para calcular a eficiência de diferentes rotas.

## 1. Configuração

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Logs de Entrega

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('logistics_data', exist_ok=True)

def generate_delivery_logs(region_id):
    num_routes = 500
    df = pd.DataFrame({
        'route_id': np.random.choice(['R1', 'R2', 'R3', 'R4', 'R5'], num_routes),
        'truck_id': np.random.randint(100, 200, num_routes),
        'delivery_time_min': np.random.normal(60, 20, num_routes),
        'distance_km': np.random.normal(40, 10, num_routes),
        'fuel_consumed_l': np.random.normal(5, 2, num_routes)
    })
    
    # Adicionar ineficiência na Rota R3
    df.loc[df['route_id'] == 'R3', 'delivery_time_min'] += 30
    
    df.to_csv(f'logistics_data/region_{region_id}.csv', index=False)

print("Gerando logs...")
for i in range(10):
    generate_delivery_logs(i)

## 3. Análise de Eficiência

In [ ]:
ddf = dd.read_csv('logistics_data/region_*.csv')

# Calcular métrica de eficiência: km por minuto
ddf['efficiency'] = ddf['distance_km'] / ddf['delivery_time_min']

# Agrupar por Rota e ver média
route_stats = ddf.groupby('route_id')[['delivery_time_min', 'efficiency']].mean()

print("Análise de Rotas:")
print(route_stats.compute())

In [ ]:
client.close()